# Modul 6 - Projektaufgabe
## Ugly is better than nothing - Generieren von Titelaufnahmen aus den METS/MODS-Daten digitalisierter Handschriften zur Übernahme in den K10plus und Nachnutzung im lokalen Nachweissystem ACTApro
### Ulrike Mehringer, 2023

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
import re

## Personen: von GND zu PPN

In [2]:
# Funktion gnd_ppn: Personennormsatz-ID (PPN) über die GND-Nummer im K10Plus abrufen

gnd_nr = "118655477"
def gnd_ppn(gnd_nr):
    
## Normsatz aus K10Plus auslesen
    url = "http://sru.k10plus.de/opac-de-627!rec=2?&operation=searchRetrieve&query=pica.nid="+gnd_nr+"&maximumRecords=1&recordSchema=mods"
    person_xml = "../data/person.xml"
    urllib.request.urlretrieve(url,person_xml)
    
## XML parsen
    person_tree = ET.parse(person_xml)
    person_root = person_tree.getroot()
    
## K10Plus-PPN aus person_xml auslesen
    rec_id = person_root.find('.//{http://www.loc.gov/mods/v3}recordIdentifier')
    
## Variable für die Textausgabe
    ppn = rec_id.text
    return(ppn)

    

## METS/MODS aus OpenDigi

In [3]:
# METS-Daten des OpenDigi-Projekts einlesen
od_projekt = "Mc339"
url = "https://idb.ub.uni-tuebingen.de/opendigi/"+od_projekt+"/mets"
od_xml = "../data/"+od_projekt+".xml"
urllib.request.urlretrieve(url,od_xml)

# Zieldatei anlegen
datei = open("../data/"+od_projekt+".txt",'a')

In [4]:
od_tree = ET.parse(od_xml)
od_root = od_tree.getroot()

In [5]:
od_data = od_root.find(".//{http://www.loc.gov/METS/}xmlData/*")

In [6]:
type(od_data)

xml.etree.ElementTree.Element

In [7]:
for child in od_data:
    print(child.tag,child.attrib)

{http://www.loc.gov/mods/v3}titleInfo {}
{http://www.loc.gov/mods/v3}identifier {'type': 'urn'}
{http://www.loc.gov/mods/v3}recordInfo {}
{http://www.loc.gov/mods/v3}identifier {'type': 'doi'}
{http://www.loc.gov/mods/v3}location {}
{http://www.loc.gov/mods/v3}language {}
{http://www.loc.gov/mods/v3}name {'authority': 'gnd', 'authorityURI': 'http://d-nb.info/gnd/', 'type': 'personal', 'valueURI': 'http://d-nb.info/gnd/119244071'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}name {'type': 'personal'}
{http://www.loc.gov/mods/v3}originInfo {}


In [8]:
# Titel, Feld 4000
od_title = od_root.find(".//{http://www.loc.gov/mods/v3}title")
print(od_title.text)
datei.write('\n4000 '+od_title.text)

Novella super quarto libro decretalium Johannis Andreae


61

In [35]:
# PIDs: URN und DOI, Felder 2050, 2051 und 4950
od_pid = od_root.findall(".//{http://www.loc.gov/mods/v3}identifier")
for i in range(0,len(od_pid)):
    if od_pid[i].attrib["type"]=="urn":
        od_urn = od_pid[i].text
    elif od_pid[i].attrib["type"]=="doi":
        od_doi = od_pid[i].text
        
print(od_urn)
print(od_doi)

datei.write('\n2050 '+od_urn)
datei.write('\n2051 '+od_doi)
datei.write('\n4950 http://nbn-resolving.de/'+od_urn+'$xR$3Volltext$534')
datei.write('\n4950 https://doi.org/'+od_doi+'$xR')

urn:nbn:de:bsz:21-dt-258226
10.20345/digitue.25822


47

In [42]:
# Signatur, Feld 4065 $a
od_shelf = od_root.find(".//{http://www.loc.gov/mods/v3}shelfLocator")
print(od_shelf.text)
datei.write('\n4065 XA-DE$cUB Tübingen$a'+od_shelf.text)

Mc 339


32

In [43]:
# Sprache, Feld 1500
od_lang = od_root.find(".//{http://www.loc.gov/mods/v3}languageTerm")
if re.search(r"; *",od_lang.text):
    od_lang = str_replace("; ","$a",od_lang.text)
print(od_lang.text)
datei.write('\n1500 '+od_lang.text)

lat


9

In [46]:
# Entstehungsort, Feld 4046 (Feld 4040?)
od_place = od_root.find(".//{http://www.loc.gov/mods/v3}placeTerm")
print(od_place.text)
if re.search(r"[oO]. *[oO.]",od_place.text):
    od_place = "[Entstehungsort nicht ermittelbar]"
print(od_place) 
datei.write('\n4046 '+od_place)

o.O.
[Entstehungsort nicht ermittelbar]


40

In [45]:
# Entstehungsdatum, Feld 1100, 1100 $n
od_date = od_root.findall(".//{http://www.loc.gov/mods/v3}dateIssued")
# od_date ist eine Liste
od_date_norm = od_date[0].text
od_date_subm = "["+od_date[1].text+"]"
if re.search(r"[oO]. *[dD].",od_date_subm):
    od_date_norm = "[1XXX]"
    od_date_subm = "[Entstehungdatum nicht ermittelbar]"
print(od_date_norm,od_date_subm)
datei.write('\n1100 '+od_date_norm+'$n'+od_date_subm)

1393 [um 1393]


21

In [48]:
# Personen, Feld 3000, 3010
od_name = od_root.findall(".//{http://www.loc.gov/mods/v3}name")
for i in range(0,len(od_name)):
    y = str(od_name[i].attrib)
    if 'valueURI' in y:
        gnd_url = od_name[i].attrib['valueURI']
        gnd_nr = str(re.findall(r'/gnd/(.*)',gnd_url))
        name_ppn = gnd_ppn(gnd_nr)
        print(name_ppn)
    else:
        od_name_display = od_root.findall(".//{http://www.loc.gov/mods/v3}displayForm")
        print(od_name_display[i].text)
        
print(name_ppn)        


080046096
Hieronymus Angeli de Esslingen [Schreiber]
Meier de Wasszneck, Ulrich [Schreiber]
080046096
